In [7]:
import lancedb
db_path = "C:\\Lee\\work\\contract\\db\\tmp\\contact_vectors.lancedb"
# --- 1. 连接/创建 LanceDB 数据库 ---
db = lancedb.connect(db_path)
print(f"LanceDB connected at: {db.uri}")

LanceDB connected at: C:\Lee\work\contract\db\tmp\contact_vectors.lancedb


In [8]:
# --- 2. 打开表 ---
table_name = "contact_table" # 你之前创建的表名
try:
    table = db.open_table(table_name)
    print(f"Successfully opened table: '{table.name}'")
    print(f"Table schema: {table.schema}")
    print(f"Number of rows in table: {len(table)}")
except Exception as e:
    print(f"Error opening table '{table_name}': {e}")
    print(f"Available tables: {db.table_names()}")
    exit()

Successfully opened table: 'contact_table'
Table schema: vector: fixed_size_list<item: float>[1024]
  child 0, item: float
id: string
payload: string
Number of rows in table: 81


In [10]:
import json

def read_and_decode_table(db, table_name):
    table = db.open_table(table_name)
    df = table.to_pandas()
    
    if 'payload' not in df.columns:
        raise ValueError("DataFrame中缺少'payload'列")
    
    # 解析payload字符串为字典
    df['parsed_payload'] = df['payload'].apply(
        lambda x: json.loads(x) if isinstance(x, str) else x
    )
    
    # 直接提取已解码的字段值
    df['name_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('name') if isinstance(x, dict) else None
    )
    
    df['content_cn'] = df['parsed_payload'].apply(
        lambda x: x.get('content') if isinstance(x, dict) else None
    )
    
    # 可选：删除临时解析列
    df = df.drop(columns=['parsed_payload'])
    
    return df

In [11]:
data = read_and_decode_table(db, table_name)

In [12]:
data.iloc[0]['content_cn']

'买方合同编号：YFYR2023\\-04\n\n卖方合同编号：\n\n订 货 合 同\n\n项目名称：山东钢铁集团永锋临港有限公司\n\n临港基地二期2\\#烧结机余热发电项目\n\n子项名称：烧结机余热锅炉\n\n买方：湖南中冶长天节能环保技术有限公司\n\n卖方：江苏中天能源设备有限公司\n\n合同签订时期：2023年4月\n\n合同签订地点：湖南·长沙\n\n合 同 双 方\n\n__买\u3000\u3000方（印章）__\n\n__卖\u3000\u3000方（印章）__\n\n__名   称：湖南中冶长天节能环保技术有限公司__\n\n__名  称：江苏中天能源设备有限公司__\n\n__地    址：长沙市岳麓区节庆路7号__\n\n__地址：江苏省扬州市江都区宜陵镇新康路1号__\n\n__联 系 人：刘璇__\n\n__联 系 人：周文韬__\n\n__电    话：18674469888__\n\n__电    话：13852746888__\n\n__传    真：__\n\n__传    真：__\n\n__电子邮件：23932814@qq\\.com__\n\n__电子邮件：13852797676@163\\.com__\n\n__开户银行：浦发银行长沙东塘支行__\n\n__开户银行：江都农村商业银行七里支行__\n\n__帐    号：6605 0155 2000 0246 8__\n\n__帐    号：3210882301201000008200__\n\n__单位名称：湖南中冶长天节能环保技术有限公司__\n\n__单位名称：江苏中天能源设备有限公司__\n\n__委托代理人：__\n\n__委托代理人： __\n\n__签字日期：__\n\n__签字日期：__\n\n3\\.1  最终供货一览表\n\n__序号__\n\n__货物名称__\n\n__规格型号__\n\n__单位__\n\n__数量__\n\n__重量（t）__\n\n__单   价__\n\n__（万元RMB）__\n\n__总   价__\n\n__（万元RMB）__\n\n__交 货 期__\n\n__交货__\n\n__地点__\n\n__交货方式__\n\n1\n\n烧结机余热锅炉\n\n套\n\n1\n\n166\\.34\n\n213\\.8\n\

In [14]:
len(data.iloc[0]['content_cn'])

1499

In [15]:
data

,vector,id,payload,name_cn,content_cn
0,"[0.010003624, -0.052496962, 0.059267655, 0.025...",231c8a5e15e551e1d5870ad2bbea990d,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：YFYR2023\-04\n\n卖方合同编号：\n\n订 货 合 同\n\n项...
1,"[0.01984362, -0.0725239, 0.05276262, 0.0152738...",65ba24dea3f708e78a78eab2f5dbf18e,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：BGGFNXTS\-2022\-05__\n\n__卖方合同编号：__\n...
2,"[0.055528764, -0.040872104, 0.03574227, 0.0064...",1bc3b6461cd689f23965c24fa1eb1b85,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：JYWFFS\-2020\-05\n\n卖方合同编号：DJ\-2034Y\n\...
3,"[0.03304358, -0.090981424, -0.0049218237, 0.03...",8b656cb675ce33ea62d8bfd406b4dfc5,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：LGYR2024\-05\n\n卖方合同编号：\n\n订 货 合 同\n\n项...
4,"[0.02714693, -0.04305296, 0.053879887, -0.0057...",08a68a39353aa71595b1cf9528a85f8d,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：JSXGYR2024\-07__\n\n__卖方合同编号：__\n\n订 ...
5,"[0.008293288, -0.047394957, 0.063790694, 0.012...",8be2a2fe4888f815b8238616d743cb36,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：MGFD\-2020\-11\n\n卖方合同编号：\n\n订 货 合 同\n\...
6,"[0.029042367, -0.0670312, 0.04527044, -0.00822...",7a16af68b447896f349687bf638d8e27,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：NGYR2022\-01__\n\n__卖方合同编号：__\n\n订 货 ...
7,"[0.028757885, -0.05789458, 0.06436589, -0.0078...",f8bf4d1677f8e3098ef8dd75d19298b3,"{""name"": null, ""meta_data"": {}, ""content"": ""__...",None,__买方合同编号：BGSS\-NH\-001__\n\n__卖方合同编号：__\n\n订货合...
8,"[0.00824705, -0.046710312, 0.05740008, 0.03134...",3dc473eb2f37f6ad3d1c4fd1faac53d8,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,山东钢铁集团永锋临港有限公司\n\n临港先进优特钢产业基地一期项目\n\n提铁减锌项目\n\...
9,"[0.027195001, -0.055842165, 0.066568345, 0.005...",de588ef3e2f54690df9697010cfdc65f,"{""name"": null, ""meta_data"": {}, ""content"": ""\u...",None,买方合同编号：MGFD5\-2021\-01\n\n卖方合同编号：\n\n订 货 合 同\n...
